# 한글 텍스트 처리 - 네이버 영화 평점 감성 분석
- KoNLPy

## 한글 NLP 처리의 어려움
- 띄어쓰기와 다양한 조사 때문

## KoNLPy 소개
- python의 대표적인 한글 형태소 패키지
    - 형태소 : 단어로서 의미를 가지는 최소 단위
    - 형태소 어근 단위로 쪼개고 각 형태소에 품사 태깅(POS tagging)을 부착하는 작업
- KoNLPy : C/C++, Java로 만들어진 형태소 엔진을 파이썬 래퍼(Wrapper) 기반으로 재작성한 패키지
    - 꼬꼬마(Kkma), 한나눔(Hannanum), Komoran, 은전한닢 프로젝트(Mecab), Okt의 형태소 모듈 사용 가능
    - Mecab의 경우 리눅스에서만 됐으나, 지금은 윈도우에서도 가능

### 설치
1. JPype1 모듈 설치
    - `conda install -c conda-forge jpype1`
    - 또는 `pip install --upgrade pip` 후 [JPype1 다운로드 링크](https://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype)에서 파이썬 버전, 운영체제가 맞는 whl 파일을 받은 뒤 `pip install 파일명.whl`을 통해 설치
        - 나의 경우, 위 방법으로는 안 되고 아래 방법으로 `JPype1-1.1.2-cp37-cp37m-win_amd64.whl` 파일 다운 후 설치
2. Java 환경 설정
    - [JDK 다운로드](https://www.oracle.com/java/technologies/javase-downloads.html) 후 설치 (나의 경우 가장 최신 버전인 16.0.2)
    - `C:\Program Files\Java\jdk-16.0.2\bin\server`에 있는 `jvm.dll` 파일에 대하여 윈도우 환경 변수 추가
        - 환경 변수 - 사용자 변수 - 변수 이름 : JAVA_HOME, 변수 값 : C:\Program Files\Java\jdk-16.0.2\bin\server - 확인
3. KoNLPy 설치
    - `pip install konlpy`

In [1]:
import sys
sys.version

'3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:11:50) [MSC v.1916 64 bit (AMD64)]'

## 네이버 영화 평점 리뷰
- [txt 파일](https://github.com/e9t/nsmc)
- label : 1=긍정, 0=부정

### 데이터 로딩

In [2]:
import pandas as pd

train_df = pd.read_csv('../data/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

### 데이터 가공
- null 값을 공백(' ')으로 대체
- 숫자의 경우 단어적인 의미가 부족하므로 공백으로 대체

In [4]:
import re

train_df = train_df.fillna(' ')

# 정규표현식 \d : 숫자
train_df['document'] = train_df['document'].apply(lambda x:re.sub(r'\d', ' ', x))

# test 데이터에 대해서도 똑같이
test_df = pd.read_csv('../data/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x:re.sub(r'\d', ' ', x))

# id 칼럼 삭제
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

### 토큰화
- 형태소 단위로 토큰화
- SNS 분석에 적합한 Okt(前 Twitter, 책에는 이걸로 되어 있음) 클래스 이용

In [7]:
from konlpy.tag import Okt

okt = Okt()
def okt_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [14]:
text_simple = train_df.loc[0, 'document']
print(text_simple, '>>', okt_tokenizer(text_simple))

아 더빙.. 진짜 짜증나네요 목소리 >> ['아', '더빙', '..', '진짜', '짜증나네요', '목소리']


In [15]:
text_simple = train_df.loc[3, 'document']
print(text_simple, '>>', okt_tokenizer(text_simple))

교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 >> ['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']


In [16]:
text_simple = train_df.loc[5, 'document']
print(text_simple, '>>', okt_tokenizer(text_simple))

막 걸음마 뗀  세부터 초등학교  학년생인  살용영화.ㅋㅋㅋ...별반개도 아까움. >> ['막', '걸음', '마', '뗀', '세', '부터', '초등학교', '학년', '생인', '살용', '영화', '.', 'ㅋㅋㅋ', '...', '별', '반개', '도', '아까', '움', '.']


### TF-IDF 벡터화 및 학습, GridSearch, 평가

In [18]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

Wall time: 8min 53s


In [19]:
%%time
lr_clf = LogisticRegression()

params = {'C':[0.5, 0.8, 1, 2, 3, 3.5, 4, 4.5, 5, 5.5, 10]}

grid_cv = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

Fitting 3 folds for each of 11 candidates, totalling 33 fits
{'C': 3} 0.8593
Wall time: 1min 26s


In [20]:
from sklearn.metrics import accuracy_score

# 테스트 데이터 변환
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# best param classifier 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)
acc = accuracy_score(test_df['label'], preds)

print('Logistic Regression 정확도 :', acc)

Logistic Regression 정확도 : 0.86182
